In [1]:
%matplotlib inline

In [2]:
from read_sfh import get_scales
small_file_fname = "/Users/aphearin/Dropbox/UniverseMachine/data/histories/small_sfh_catalog_1.002310.txt"
scale_factor_array = get_scales(small_file_fname)
cosmic_age_array = np.load('/Users/aphearin/Dropbox/UniverseMachine/data/histories/prelim_sfh_reduction/times.npy')

In [4]:
from collate_full_catalog import collate_catalog
a_string = 'a_1.002310'
idx_tobs = np.where(scale_factor_array >= float(a_string[2:]))[0][0]
t_today = cosmic_age_array[idx_tobs]

history_colnames = ('halo_id', 'halo_upid', 'halo_mpeak', 'sfr_mp', 'sm_mp', 
                    'stellar_mass', 'sfr', 'a_first_infall', 'halo_mvir', 'halo_vmax')

histories = collate_catalog(a_string, history_colnames=history_colnames)
print("Number of galaxies = {0}".format(len(histories)))

... Assembling history data
Number of galaxies = 676028


In [10]:
from halotools.utils import crossmatch

subhalo_mask = histories['halo_upid'] != -1
subs = histories[subhalo_mask]
idxA, idxB = crossmatch(histories['halo_upid'][subhalo_mask], histories['halo_id'])

histories['host_mvir'] = histories['halo_mvir']
histories['host_mvir'][subhalo_mask][idxA] = histories['halo_mvir'][idxB]
histories['host_vmax'] = histories['halo_vmax']
histories['host_vmax'][subhalo_mask][idxA] = histories['halo_vmax'][idxB]

In [28]:
from scipy.stats import binned_statistic
cenmask = (histories['gal_type'] == 'central') | (histories['gal_type'] == 'backsplash')
median_log_vmax, log_mvir_bin_edges, __ = binned_statistic(np.log10(histories['halo_mvir'][cenmask]), 
                 np.log10(histories['halo_vmax'][cenmask]), statistic='median')
log_mvir_bin_mids = 0.5*(log_mvir_bin_edges[1:] + log_mvir_bin_edges[:-1])

interpolated_median_log_vmax = np.interp(np.log10(histories['host_mvir']), 
                                         log_mvir_bin_mids, median_log_vmax)
histories['high_host_vmax'] = True
histories['high_host_vmax'][histories['host_vmax'] < 10**interpolated_median_log_vmax] = False